<a href="https://colab.research.google.com/github/svplaksin/simulative-python/blob/main/%D0%92%D1%82%D0%BE%D1%80%D0%BE%D0%B9_%D0%BA%D0%B5%D0%B9%D1%81.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 2 кейс

**Соберите данные из Google Sheets и напишите функцию `generate_report`, которая принимает три списка (данные с каждого листа в Google Sheets) и сохраняет отчет в файл `student_debt_report.txt`**

**Важно**

Перед началом решения задачи выполните следующую ячейку - в ней скачиваются нужные файлы

In [1]:
!wget https://gist.github.com/Vs8th/d0bd4bdbbb58c8ae4f70a2a503e2d5fc/raw/creds.json

!wget https://gist.github.com/Vs8th/39c5deed0f5539d781f00328f7fd4fe0/raw/result.txt

--2025-08-17 05:21:31--  https://gist.github.com/Vs8th/d0bd4bdbbb58c8ae4f70a2a503e2d5fc/raw/creds.json
Resolving gist.github.com (gist.github.com)... 20.27.177.113
Connecting to gist.github.com (gist.github.com)|20.27.177.113|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://gist.githubusercontent.com/Vs8th/d0bd4bdbbb58c8ae4f70a2a503e2d5fc/raw/creds.json [following]
--2025-08-17 05:21:31--  https://gist.githubusercontent.com/Vs8th/d0bd4bdbbb58c8ae4f70a2a503e2d5fc/raw/creds.json
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.108.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2358 (2.3K) [text/plain]
Saving to: ‘creds.json’

creds.json          100%[===================>]   2.30K  --.-KB/s    in 0s      

2025-08-17 05:21:31 (30.2 MB/s) - ‘creds.json’ saved 

Чтобы посмотреть как выглядят сами гугл таблицы в оригинале - можете перейти по [ссылке](https://docs.google.com/spreadsheets/d/1hRnw-PEftF0J-6KY7InlILVwWdkJu4vJiGwRjuE_P4U/edit?usp=sharing) и изучить.

Небольшое описание столбцов в таблицах:  
**Лист1:** `student_id` - айди студента; `student_name` - имя студента; `installment` (Y/N) - факт наличия рассрочки у студента (Y - рассрочка есть, N - рассрочки нет).  
**Лист2:** `student_id` - айди студента; `last_payment_date` - дата последней полученной оплаты; `expected_payment_date` - дата, в которую ожидаем следующий платеж (рассчитывается от даты последнего платежа).  
**Лист3:** `student_id` - айди студента; `already_payed_amount` - уже выплаченная часть рассрочки; `left_to_pay` - оставшаяся (невыплаченная) часть; `one-time_payment` - единоразовый платеж; `installment_amount` - общая сумма, которая бралась в рассрочку.

Как примерно должен выглядеть результат:

In [2]:
with open('result.txt') as f:
  print(f.read())

Студент Иванов У.У. - долг 100000 рублей
Студент Петров Е.Е. - долг 250000 рублей
Студент Сидоров И.И. - долг 10000 рублей


In [ ]:
#@title Если возникнут трудности с определением `scope`, подсказка:

scope = ['https://www.googleapis.com/auth/spreadsheets.readonly',
         'https://www.googleapis.com/auth/drive']

**Примечание**

Считать должников необходимо на 1 марта 2023 года. То есть определяя количество просроченных платежей, мы определяем их количество не на настоящую дату, а на **1 марта 2023 года**. А периоды внесения платежей для всех студентов одинаковы - **6 месяцев** (183 дня).

То есть, если ожидаемый платеж должен был пройти 3 февраля 2022 года, то к 1 марта 2023 студент просрочил уже три платежа (3 февраля 2022, 3 августа 2022 и 3 февраля 2023). При расчетах отталкивайтесь от даты ожидаемого платежа, разницу между платежами берите 183 дня.

**Решение**

Напишите свое решение ниже

In [145]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from datetime import timedelta, datetime, date
# Указываем необходимые права доступа к таблицам
scope = ['https://www.googleapis.com/auth/spreadsheets.readonly',
         'https://www.googleapis.com/auth/drive']

# Загружаем ключи аутентификации из файла json
creds = ServiceAccountCredentials.from_json_keyfile_name('creds.json', scope)

# Авторизуемся в Google Sheets API
client = gspread.authorize(creds)

worksheet = client.open_by_key("1hRnw-PEftF0J-6KY7InlILVwWdkJu4vJiGwRjuE_P4U")

In [146]:
TARGET_DATE = date(2023, 3, 1) # Смотрим должников на 01.03.2023
# Формируем списки для формирования отчета
names, dates, payments = [lst.get_all_values() for lst in worksheet.worksheets()]

def generate_report(names, dates, payments):
  # Создаем словарь {id клиента: ФИО}
  names = {int(name[0]): name[1] for name in names[1:] if name[-1] == 'Y'}
  # Формируем словарь {id клиента: Дата ожидаемого платежа} для должников
  dates = {int(day[0]): datetime.strptime(day[2], '%d.%m.%Y').date()
           for day in dates[1:] # Убираем заголовки
           if datetime.strptime(day[2], '%d.%m.%Y').date() < date(2023, 3, 1)} # Если дата < 01.03.2023,
  # Формируем словарь с общей суммой долга и единовременным платежом
  payments = {int(pay[0]): {'left_to_pay': int(pay[2]),
                            'one-time_payment': int(pay[3])
                            }
              for pay in payments[1:] # Убираем заголовки
              if int(pay[0]) in dates.keys()} # Только для должников
  # Формируем словарь с количеством просроченных периодов для каждого должника
  periods = {k:(TARGET_DATE - dates[k]).days // 183 + 1 for k in dates.keys()}
  # Формируем список строк с долгами.
  # Размер долга - это минимум из общей суммы долга и произведением количества
  # просроченных периодов на размер единовременного платежа
  report_list = [f"Студент {names[i]} - долг {min(payments[i]['left_to_pay'], periods[i] * payments[i]['one-time_payment'])} рублей" for i in dates.keys()]

  with open('student_debt_report.txt', 'w') as f:
    f.writelines('\n'.join(report_list))

In [147]:
generate_report(names, dates, payments)

✏️ ✏️ ✏️

**Проверка**

Чтобы проверить свое решение, запустите код в следующих ячейках

In [148]:
# Здесь будет скачиваться файл с эталонным ответом

!wget https://gist.github.com/Vs8th/63832f093f4db8d8b251ba5d39571f3d/raw/student_debt.txt

import pandas as pd

user_answer = pd.read_csv('student_debt_report.txt')
correct_answer = pd.read_csv('student_debt.txt')

--2025-08-17 10:53:35--  https://gist.github.com/Vs8th/63832f093f4db8d8b251ba5d39571f3d/raw/student_debt.txt
Resolving gist.github.com (gist.github.com)... 20.27.177.113
Connecting to gist.github.com (gist.github.com)|20.27.177.113|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://gist.githubusercontent.com/Vs8th/63832f093f4db8d8b251ba5d39571f3d/raw/student_debt.txt [following]
--2025-08-17 10:53:35--  https://gist.githubusercontent.com/Vs8th/63832f093f4db8d8b251ba5d39571f3d/raw/student_debt.txt
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.108.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11325 (11K) [text/plain]
Saving to: ‘student_debt.txt.1’

student_debt.txt.1  100%[===================>]  11.06K  --.-KB/s    in 0.001s  

2025-08-17 10:53:35 (13.0 M

In [149]:
try:
  assert (user_answer == correct_answer).all().all(), 'Ответы не совпадают'
  assert user_answer.columns.equals(correct_answer.columns), 'Названия столбцов не совпадают'
except Exception as err:
  raise AssertionError(f'При проверке возникла ошибка {repr(err)}')
else:
  print('Поздравляем, Вы справились и успешно прошли все проверки!!')

Поздравляем, Вы справились и успешно прошли все проверки!!
